In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

os.chdir("../../../../")

import yaml
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
import mudata as md
import muon as mu
import numpy as np
import scanpy as sc
from matplotlib import gridspec
import scanpy.external as sce
from scipy import sparse
from matplotlib.colors import LinearSegmentedColormap

from utils.utils import *
from utils.plotting import *

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=100,
    facecolor="white",
    # color_map="YlGnBu",
    frameon=False,
)
import matplotlib as mpl

sc.set_figure_params(dpi=100)
sns.set(style="white")

In [ ]:
path = "data/anca/exploratory/ANCA_27PK27PB_T_harmony_r.h5mu"
mudata = md.read_h5mu(path)

mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]

In [ ]:
sc.pl.umap(mod_rna, color="cell_type_yu")

In [ ]:
sc.pl.umap(
    mod_rna,
    color=["sample", "tissue", "patient"],
    # return_fig=True,
    # legend_loc="on data",
    # save="_leiden.png",
    wspace=0.8,
)

In [ ]:
plot_leiden_results(mod_rna, rep_name="")

In [ ]:
run_celltypist(mod_rna)
sc.pl.umap(
    mod_rna, color=["celltypist_cell_label_coarse", "celltypist_cell_label_fine"]
)

In [ ]:
cluster_key = "leiden_1.7"

In [ ]:
all_marker_results, filtered_marker_results, best_markers = run_de_pipeline(
    mod_rna,
    cluster_key,
    mod="rna",
    top_n=5,
    min_expression=0.1,
)

In [ ]:
top_markers = get_top_n_markers(best_markers, n=5)
# top_markers = {k: top_markers[k] for k in cell_type_order}
sc.pl.dotplot(
    mod_rna,
    var_names=top_markers,
    groupby=cluster_key,
    standard_scale="var",
    # color_map="Reds",
    # swap_axes=True,
    var_group_rotation=25,
)

In [ ]:
marker_df = pd.read_excel("data/markers/T cell markers.xlsx")
marker_df = marker_df.set_index("Cell type")

# extract positive markers
marker_db = {}
for index, row in marker_df.iterrows():
    pos_markers_1 = (
        row["positive markers 1"].replace(" ", "").split(",")
        if row["positive markers 1"] is not np.nan
        else []
    )
    pos_markers_2 = (
        row["positive markers 2"].replace(" ", "").split(",")
        if row["positive markers 2"] is not np.nan
        else []
    )
    marker_db[index] = list(set(pos_markers_1 + pos_markers_2))

marker_plotter = MarkerPlotter(mod_rna, mod_cite, marker_db, cluster_key=cluster_key)

In [ ]:
fig = marker_plotter.visualize_markers(
    markers=["LYZ", "CD14", "APOBEC3A", "CFP", "S100A9", "S100A8", "CD16", "PTPRC"],
    dtype="rna",
    ncols=3,
    use_default_plot=True,
    # marker_plot=False
    # dotplot=False,
)

In [ ]:
fig = marker_plotter.visualize_markers(
    markers=[
        "CD8A",
        "CD4",
        "CD3E",
    ],
    dtype="rna",
    ncols=3,
    use_default_plot=True,
    # marker_plot=False
    # dotplot=False,
)

## RNA markers

In [ ]:
for cell_type in marker_db.keys():
    fig = marker_plotter.visualize_markers(
        cell_type=cell_type,
        dtype="rna",
        ncols=3,
        use_default_plot=True,
        # marker_plot=False
        # dotplot=False,
    )

In [ ]:
fig = marker_plotter.visualize_markers(
    markers=["TRGV2", "TRGV4", "TRGV9", "TRDV2", "TRDV1", "TRDC"],
    dtype="rna",
    ncols=3,
    use_default_plot=True,
    # marker_plot=False
    # dotplot=False,
)

## Protein markers

In [ ]:
mod_cite.obsm["X_umap"] = mod_rna.obsm["X_umap"].copy()
mod_cite.obs[cluster_key] = mod_rna.obs[cluster_key].copy()

In [ ]:
fig = marker_plotter.visualize_markers(
    markers=["LYZ", "CD14", "APOBEC3A", "CFP", "S100A9", "S100A8", "CD16", "CD45"],
    dtype="protein",
    ncols=3,
    use_default_plot=True,
    # marker_plot=False
    # dotplot=False,
)

In [ ]:
for cell_type in marker_db.keys():
    fig = marker_plotter.visualize_markers(
        cell_type=cell_type,
        dtype="protein",
        ncols=3,
        use_default_plot=True,
        # marker_plot=False
        dotplot=False,
    )